In [8]:
!pip install torch torchvision pandas numpy scikit-learn matplotlib seaborn tqdm pillow transformers


In [10]:
!pip uninstall tensorflow tensorflow-gpu -y


Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0


In [11]:
!pip install tensorflow


  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)


In [18]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

from transformers import ViTForImageClassification, ViTImageProcessor


ImportError: Traceback (most recent call last):
  File "d:\code\.venv\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [17]:
from transformers import ViTForImageClassification, ViTImageProcessor

ImportError: Traceback (most recent call last):
  File "d:\code\.venv\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [12]:
# Part 1: Import Libraries and Setup
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from transformers import ViTForImageClassification, ViTImageProcessor
from transformers import TrainingArguments, Trainer
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

ImportError: Traceback (most recent call last):
  File "d:\code\.venv\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [13]:


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

Using device: cpu


In [14]:
# Part 2: Dataset Configuration and Custom Dataset Class

# Configuration - UPDATE THESE PATHS TO YOUR ACTUAL PATHS
CONFIG = {
    'schizophrenia_folder': 'D:/HHT/S',  # Update this path
    'healthy_folder': 'D:/HHT/H',              # Update this path
    'batch_size': 16,
    'num_epochs': 50,
    'learning_rate': 2e-5,
    'image_size': 224,
    'num_folds': 5,
    'num_classes': 2,
    'model_name': 'google/vit-base-patch16-224-in21k'
}

class EEGDataset(Dataset):
    def __init__(self, image_paths, labels, processor, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        # Apply transforms if provided
        if self.transform:
            image = self.transform(image)
        
        # Process with ViT processor
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)
        
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return {
            'pixel_values': pixel_values,
            'labels': label
        }

# Data augmentation transforms
train_transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
])

val_transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
])

print("Dataset class and transforms defined successfully!")

Dataset class and transforms defined successfully!


In [16]:
# Part 3: Data Loading and Preparation

def load_data_paths(schizophrenia_folder, healthy_folder):
    """Load image paths and create labels"""
    image_paths = []
    labels = []
    
    # Load schizophrenia images (label = 1)
    if os.path.exists(schizophrenia_folder):
        schiz_files = [f for f in os.listdir(schizophrenia_folder) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for file in schiz_files:
            image_paths.append(os.path.join(schizophrenia_folder, file))
            labels.append(1)
        print(f"Loaded {len(schiz_files)} schizophrenia images")
    else:
        print(f"Warning: Schizophrenia folder not found at {schizophrenia_folder}")
    
    # Load healthy images (label = 0)
    if os.path.exists(healthy_folder):
        healthy_files = [f for f in os.listdir(healthy_folder) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for file in healthy_files:
            image_paths.append(os.path.join(healthy_folder, file))
            labels.append(0)
        print(f"Loaded {len(healthy_files)} healthy images")
    else:
        print(f"Warning: Healthy folder not found at {healthy_folder}")
    
    return image_paths, labels

# Load the data
print("Loading data paths...")
image_paths, labels = load_data_paths(CONFIG['schizophrenia_folder'], CONFIG['healthy_folder'])

print(f"Total images loaded: {len(image_paths)}")
print(f"Class distribution:")
print(f"  Healthy (0): {labels.count(0)}")
print(f"  Schizophrenia (1): {labels.count(1)}")

# Convert to numpy arrays for easier handling
image_paths = np.array(image_paths)
labels = np.array(labels)

# Initialize ViT processor
processor = ViTImageProcessor.from_pretrained(CONFIG['model_name'])
print("ViT processor initialized successfully!")

Loading data paths...
Loaded 5146 schizophrenia images
Loaded 4235 healthy images
Total images loaded: 9381
Class distribution:
  Healthy (0): 4235
  Schizophrenia (1): 5146


NameError: name 'ViTImageProcessor' is not defined

In [6]:
# Part 2: Data Loading and Preprocessing

def load_image_paths_and_labels(schizo_folder, healthy_folder):
    """Load image paths and create labels"""
    image_paths = []
    labels = []
    
    # Load schizophrenia images (label = 1)
    if os.path.exists(schizo_folder):
        schizo_images = [f for f in os.listdir(schizo_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for img in schizo_images:
            image_paths.append(os.path.join(schizo_folder, img))
            labels.append(1)
        print(f"Loaded {len(schizo_images)} schizophrenia images")
    
    # Load healthy images (label = 0)
    if os.path.exists(healthy_folder):
        healthy_images = [f for f in os.listdir(healthy_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for img in healthy_images:
            image_paths.append(os.path.join(healthy_folder, img))
            labels.append(0)
        print(f"Loaded {len(healthy_images)} healthy images")
    
    return image_paths, labels

class EEGImageDataset(Dataset):
    def __init__(self, image_paths, labels, processor, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        # Apply additional transforms if provided
        if self.transform:
            image = self.transform(image)
        
        # Process with ViT processor
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)
        
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return pixel_values, label

# Load data
print("Loading image paths and labels...")
image_paths, labels = load_image_paths_and_labels(
    CONFIG['schizophrenia_folder'], 
    CONFIG['healthy_folder']
)

print(f"Total images loaded: {len(image_paths)}")
print(f"Class distribution: Healthy={labels.count(0)}, Schizophrenia={labels.count(1)}")

# Initialize ViT processor
processor = ViTImageProcessor.from_pretrained(CONFIG['model_name'])

# Data augmentation transforms
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
])

val_transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
])

print("Data loading and preprocessing setup complete!")

Loading image paths and labels...
Loaded 5146 schizophrenia images
Loaded 4235 healthy images
Total images loaded: 9381
Class distribution: Healthy=4235, Schizophrenia=5146
Data loading and preprocessing setup complete!


In [7]:
# Part 3: Model Definition

class ViTClassifier(nn.Module):
    def __init__(self, model_name, num_classes, dropout_rate=0.1):
        super(ViTClassifier, self).__init__()
        self.vit = ViTForImageClassification.from_pretrained(
            model_name,
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )
        
        # Add dropout for regularization
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, pixel_values):
        outputs = self.vit(pixel_values=pixel_values)
        return outputs.logits

def create_model():
    """Create and initialize the ViT model"""
    model = ViTClassifier(
        model_name=CONFIG['model_name'],
        num_classes=CONFIG['num_classes']
    )
    return model

# Test model creation
print("Testing model creation...")
test_model = create_model()
print(f"Model created successfully!")
print(f"Model parameters: {sum(p.numel() for p in test_model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in test_model.parameters() if p.requires_grad):,}")

# Move to device
test_model = test_model.to(device)
print(f"Model moved to {device}")

del test_model  # Clean up memory
torch.cuda.empty_cache() if torch.cuda.is_available() else None

Testing model creation...


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model created successfully!
Model parameters: 85,800,194
Trainable parameters: 85,800,194
Model moved to cpu


In [8]:
# Part 4: Training and Validation Functions

def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train model for one epoch"""
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    for batch_idx, (pixel_values, labels) in enumerate(train_loader):
        pixel_values = pixel_values.to(device)
        labels = labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(pixel_values)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
        if batch_idx % 10 == 0:
            print(f'Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_predictions / total_predictions
    
    return epoch_loss, epoch_acc

def validate_epoch(model, val_loader, criterion, device):
    """Validate model for one epoch"""
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for pixel_values, labels in val_loader:
            pixel_values = pixel_values.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(pixel_values)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            
            # Store predictions and labels for confusion matrix
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = correct_predictions / total_predictions
    
    return epoch_loss, epoch_acc, all_predictions, all_labels

def save_model(model, fold, accuracy, save_dir='models'):
    """Save the best model"""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    model_path = os.path.join(save_dir, f'best_vit_model_fold_{fold}_acc_{accuracy:.4f}.pth')
    torch.save({
        'model_state_dict': model.state_dict(),
        'accuracy': accuracy,
        'fold': fold,
        'config': CONFIG
    }, model_path)
    
    return model_path

print("Training and validation functions defined!")

Training and validation functions defined!


In [ ]:
# Part 5: 5-Fold Cross-Validation Training

def run_cross_validation():
    """Run 5-fold cross-validation training"""
    
    # Convert to numpy arrays for sklearn
    image_paths_array = np.array(image_paths)
    labels_array = np.array(labels)
    
    # Initialize stratified k-fold
    skf = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=42)
    
    # Store results
    fold_results = []
    all_fold_predictions = []
    all_fold_labels = []
    best_models = []
    
    print(f"Starting {CONFIG['num_folds']}-fold cross-validation...")
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths_array, labels_array)):
        print(f"\n" + "="*50)
        print(f"FOLD {fold + 1}/{CONFIG['num_folds']}")
        print("="*50)
        
        # Split data
        train_paths = image_paths_array[train_idx].tolist()
        train_labels = labels_array[train_idx].tolist()
        val_paths = image_paths_array[val_idx].tolist()
        val_labels = labels_array[val_idx].tolist()
        
        print(f"Train samples: {len(train_paths)}, Val samples: {len(val_paths)}")
        
        # Create datasets
        train_dataset = EEGImageDataset(train_paths, train_labels, processor, train_transform)
        val_dataset = EEGImageDataset(val_paths, val_labels, processor, val_transform)
        
        # Create data loaders
        train_loader = DataLoader(
            train_dataset, 
            batch_size=CONFIG['batch_size'], 
            shuffle=True, 
            num_workers=2
        )
        val_loader = DataLoader(
            val_dataset, 
            batch_size=CONFIG['batch_size'], 
            shuffle=False, 
            num_workers=2
        )
        
        # Create model
        model = create_model().to(device)
        
        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=0.01)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5, factor=0.5)
        
        # Training variables
        best_val_acc = 0.0
        best_model_path = None
        train_losses = []
        train_accs = []
        val_losses = []
        val_accs = []
        
        print(f"Training fold {fold + 1}...")
        
        for epoch in range(CONFIG['num_epochs']):
            print(f"\nEpoch {epoch + 1}/{CONFIG['num_epochs']}")
            print("-" * 30)
            
            # Train
            train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
            
            # Validate
            val_loss, val_acc, val_predictions, val_true_labels = validate_epoch(
                model, val_loader, criterion, device
            )
            
            # Update learning rate
            scheduler.step(val_acc)
            
            # Store metrics
            train_losses.append(train_loss)
            train_accs.append(train_acc)
            val_losses.append(val_loss)
            val_accs.append(val_acc)
            
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            # Save best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model_path = save_model(model, fold + 1, val_acc)
                print(f"New best model saved: {best_model_path}")
        
        # Store fold results
        fold_result = {
            'fold': fold + 1,
            'best_val_acc': best_val_acc,
            'best_model_path': best_model_path,
            'train_losses': train_losses,
            'train_accs': train_accs,
            'val_losses': val_losses,
            'val_accs': val_accs,
            'final_predictions': val_predictions,
            'final_true_labels': val_true_labels
        }
        
        fold_results.append(fold_result)
        all_fold_predictions.extend(val_predictions)
        all_fold_labels.extend(val_true_labels)
        
        print(f"Fold {fold + 1} completed. Best validation accuracy: {best_val_acc:.4f}")
        
        # Clean up memory
        del model
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return fold_results, all_fold_predictions, all_fold_labels

# Run cross-validation
print("Starting cross-validation training...")
cv_results, all_predictions, all_true_labels = run_cross_validation()

Starting cross-validation training...
Starting 5-fold cross-validation...

FOLD 1/5
Train samples: 7504, Val samples: 1877


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training fold 1...

Epoch 1/50
------------------------------
